In [35]:
from sqlalchemy import create_engine
import pandas as pd

In [36]:
# create path to data source
csv = 'rawdata/data.csv'

In [67]:
# use pandas read_csv() function to read in data
p = pd.read_csv(csv)
# extract only the columns we want to use
gym = p[['timestamp','number_people','date','temperature']]
# rename columns and create an id column (didn't work with timestamp so i created an id column)
crowd = gym.rename(columns={'timestamp':'id',
                           'number_people':'members_workingout',
                           'date':'date',
                           'temperature':'temp'})
# code to create id column off of index
crowd["id"] = crowd.index + 1
# dropping duplicates in the dataset
crowd.drop_duplicates(inplace=True)
# see what type of data each column is for pgAdmin
crowd.dtypes

id                      int64
members_workingout      int64
date                   object
temp                  float64
dtype: object

In [68]:
# create connection to pgAdmin
protocol = 'postgresql'
username = 'postgres'
password = 'postgres'
host = 'localhost'
port = 5433
database_name = 'project2'
rds_connection_string = f'{protocol}://{username}:{password}@{host}:{port}/{database_name}'
engine = create_engine(rds_connection_string)

In [69]:
# check to see your table in pgAdmin is linked to here
engine.table_names()

/Users/benraseman/opt/anaconda3/envs/PythonData/lib/python3.7/site-packages/ipykernel_launcher.py:1: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  """Entry point for launching an IPython kernel.


['crowd']

In [70]:
# load data into pgAdmin
crowd.to_sql(name='crowd',con=engine,if_exists='append',index=False)